# Balls and Bins
--------

In this notebook, We will examine the famous [Balls and Bins](https://en.wikipedia.org/wiki/Balls_into_bins) problem. Consider $n$ balls droping into $n$ bins independetly uniform at random. The theorem says: The bin with maximum load has a.a.s $\frac{(2+o(1)) \lg n}{\lg\lg n} $ balls inside.

There is another version of the problem, mainly known as The Power of Two Choices , in which each balls has picks two bins indepent of other balls and drop into the bin with smaller balls. In this case the maximum load drop dramatically into $\lg \lg n + \theta(1)$.

**Remark : ** We will plot semi-log diagram of maximum load per $n$. As $n$ must grow exponentially, we may barely reach 22 iterations. You may compare the asymptotic function with the experimental values.


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact
from ipywidgets.widgets import IntSlider,Checkbox,fixed
%matplotlib inline
plt.rcParams['figure.figsize']=(14,4)

maxx=50

This function returns maximum load for different values of balls and bins. You may see the histogram by the checkbox.

In [3]:
@interact (bins=IntSlider(min=2,max=maxx),disp=Checkbox(False,description='Display Histogram'))
def couple(bins,disp):
    realiz=np.random.randint(0,bins,maxx)
        
    @interact(balls=IntSlider(min=0,max=maxx),disp=fixed(disp))
    def balls_and_bins (balls,disp):
        A=np.zeros(bins)
        for j in range(balls):
            A[realiz[j]]+=1
        if disp: plt.bar(range(bins),A)
        return np.max(A)

interactive(children=(IntSlider(value=2, description='bins', max=50, min=2), Checkbox(value=False, description='Display Histogram'), Output()), _dom_classes=('widget-interact',))

In [4]:
@interact (bins=IntSlider(min=2,max=maxx),disp=Checkbox(False,description='Display Histogram'))
def couple(bins,disp):
    realiz=np.random.randint(0,bins,[maxx,2])

    @interact(balls=IntSlider(min=0,max=maxx),disp=fixed(disp))
    def power_of_two_choices(balls,disp=False):
        A=np.zeros(bins)
        for j in range(balls):
            idx=realiz[j,:]
            if A[idx[0]]<A[idx[1]]:
                A[idx[0]]+=1
            else:
                A[idx[1]]+=1
        if disp: plt.bar(range(bins),A)
        return np.max(A)

interactive(children=(IntSlider(value=2, description='bins', max=50, min=2), Checkbox(value=False, description='Display Histogram'), Output()), _dom_classes=('widget-interact',))

For large $n$ it is not feasible to create a huge array to store the histogram of it. There is a better way for simulation and that is to store just the histogram. Every ball that is going to drop in a bin with $i$ balls inside, increase the number of bins with $i+1$ balls and decrease the number of bins with $i$ balls. So, it is enough to just choose an $i-$bin with the probability of $n_i/\sum(n_i)$ where $n_i$ is the number of $i-$bins. This algorithm is a slower but can maintain space usage. It is also easy to generalize this algorithm for power of two choices version.

**Remark : ** You may use it in the monte carlo instead of balls_and_bins function

In [6]:
@interact (bins=IntSlider(min=2,max=maxx),power_of_two=Checkbox(False,description='Power of Two Choices'),disp=Checkbox(False,description='Display Histogram'))
def couple(bins,power_of_two,disp):
    realiz=np.random.rand(maxx,2)

    @interact(balls=IntSlider(min=5,max=maxx),power_of_two=fixed(power_of_two),disp=fixed(disp))
    def space_efficient_algorithm(balls,power_of_two,disp=False):
        bin_hist=np.zeros(balls)
        bin_hist[0]=bins
        for i in range(balls):
            cumsum=np.cumsum(bin_hist)/bins
            threshold=realiz[i,0]
            if power_of_two: threshold=min(threshold,realiz[i,1])
            choice=np.min(np.argwhere(cumsum>=threshold))
            bin_hist[choice]-=1
            bin_hist[choice+1]+=1
        maxload=np.max(np.argwhere(bin_hist>0))
        if disp:
            plt.bar(range(maxload+2),bin_hist[:maxload+2])
            plt.ylim([0,max(10,max(bin_hist))+1])
        return maxload

interactive(children=(IntSlider(value=2, description='bins', max=50, min=2), Checkbox(value=False, description='Power of Two Choices'), Checkbox(value=False, description='Display Histogram'), Output()), _dom_classes=('widget-interact',))